In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from os import listdir
import glob
from PIL import Image
import cv2
from random import randrange
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
ori_images = 'Dataset/Train/Images'
mask_images = 'Dataset/Train/Masks'

In [4]:
def load_data(dataset_path, split=0.2):
    images = [f for f in glob.glob(ori_images + "**/*.png", recursive=True)]
    masks = [f for f in glob.glob(mask_images + "**/*.png", recursive=True)]

    test_size = int(len(images) * split)

    train_x, test_x = train_test_split(images, test_size=test_size, random_state=42)
    train_y, test_y = train_test_split(masks, test_size=test_size, random_state=42)

    return (train_x, train_y), (test_x, test_y)

In [5]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)  
    x = cv2.resize(x, (128, 128))
    x = x/255.0
    x = x.astype(np.float32)
    return x 

In [6]:
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  
    x = cv2.resize(x, (128, 128))
    x = x/255.0
    x = x.astype(np.float32)                    
    x = np.expand_dims(x, axis=-1)              
    return x

In [7]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([128, 128, 3])
    y.set_shape([128, 128, 1])
    return x, y

def tf_dataset(X, Y, batch):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

In [8]:
dataset_path = "Dataset"
(train_x, train_y), (test_x, test_y) = load_data(dataset_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Test: {len(test_x)} - {len(test_y)}")

Train: 120 - 120
Test: 30 - 30


In [9]:
batch = 4
train_dataset = tf_dataset(train_x, train_y, batch)
valid_dataset = tf_dataset(test_x, test_y, batch)

In [10]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 5, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 8, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (8, 8), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    """ Encoder """
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 256)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    """ Bridge """
    b1 = conv_block(p4, 1024)

    """ Decoder """
    d1 = decoder_block(b1, s4, 32)
    d2 = decoder_block(d1, s3, 64)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 256)

    """ Outputs """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    """ Model """
    model = Model(inputs, outputs)
    return model

In [11]:
model = build_unet((128, 128, 3))
model.compile(loss="binary_crossentropy", optimizer=Adam(), metrics=['accuracy'])
model.summary()



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 128, 128, 64)         4864      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 128, 128, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 128, 128, 64)         0         ['batch_normalization[0]

 max_pooling2d_3 (MaxPoolin  (None, 8, 8, 512)            0         ['activation_7[0][0]']        
 g2D)                                                                                             
                                                                                                  
 conv2d_8 (Conv2D)           (None, 8, 8, 1024)           1310822   ['max_pooling2d_3[0][0]']     
                                                          4                                       
                                                                                                  
 batch_normalization_8 (Bat  (None, 8, 8, 1024)           4096      ['conv2d_8[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 activation_8 (Activation)   (None, 8, 8, 1024)           0         ['batch_normalization_8[0][0]'
          

                                                                                                  
 activation_15 (Activation)  (None, 64, 64, 128)          0         ['batch_normalization_15[0][0]
                                                                    ']                            
                                                                                                  
 conv2d_transpose_3 (Conv2D  (None, 128, 128, 256)        2097408   ['activation_15[0][0]']       
 Transpose)                                                                                       
                                                                                                  
 concatenate_3 (Concatenate  (None, 128, 128, 320)        0         ['conv2d_transpose_3[0][0]',  
 )                                                                   'activation_1[0][0]']        
                                                                                                  
 conv2d_16

In [12]:
model_path = 'MANUAL.h5'

In [13]:
batch_size = 16
num_epoch = 40

In [ ]:
model.fit(
    train_dataset.repeat(),
    epochs=num_epoch,
    validation_data=valid_dataset.repeat(),
    steps_per_epoch=len(train_x)//batch_size,
    validation_steps=len(test_x)//batch_size,
    
)

Epoch 1/40


7/7 [==============================] - 87s 12s/step - loss: 1.3490 - accuracy: 0.5619 - val_loss: 2019773.3750 - val_accuracy: 0.9728
Epoch 2/40
7/7 [==============================] - 87s 13s/step - loss: 0.5773 - accuracy: 0.8687 - val_loss: 368644.1250 - val_accuracy: 0.0383
Epoch 3/40
7/7 [==============================] - 91s 13s/step - loss: 0.4016 - accuracy: 0.9293 - val_loss: 783.3311 - val_accuracy: 0.8404
Epoch 4/40
7/7 [==============================] - 93s 13s/step - loss: 0.2868 - accuracy: 0.9496 - val_loss: 35.7376 - val_accuracy: 0.9692
Epoch 5/40
7/7 [==============================] - 101s 15s/step - loss: 0.2469 - accuracy: 0.9430 - val_loss: 173.4176 - val_accuracy: 0.9251
Epoch 6/40
7/7 [==============================] - 111s 16s/step - loss: 0.2472 - accuracy: 0.9348 - val_loss: 0.5073 - val_accuracy: 0.6911
Epoch 7/40
7/7 [==============================] - 96s 14s/step - loss: 0.2409 - accuracy: 0.9359 - val_loss: 119.4133 - val_accuracy: 0.3043
Epoch

In [ ]:
def graph():
    #Plot training & validation accuracy values
    plt.plot(model.history.history['accuracy'])
    plt.plot(model.history.history['loss'])
    plt.title('Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'loss'], loc='upper left')
    plt.show()
graph()

In [ ]:
model.save("BENIGN.h5")